# Reranker 최적화
### 절차
- 원본 데이터셋 기준으로, first-stage retriever 단계에서 최고 성능 조합을 고정하고 reranker 최적화 실험을 수행함
- 고정된 retriever 파라미터:
    - k: 20
    - alpha: 40
    - morphological_analyzer: bm25_kiwi_pos
    - score_threshold: 0.1

### 실험 대상 Reranker
- Cohere Reranker (v3.5)
    - https://docs.cohere.com/docs/rerank-overview
- Flash Reranker: Lightweight pairwise reranker  
    - GitHub: https://github.com/PrithivirajDamodaran/FlashRank
- Jina Reranker: jina-reranker-v2-base-multilingual  
    - HuggingFace: https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual

### 실험 결과 요약
- Cohere Reranker가 가장 높은 성능(NDCG, MAP 기준)을 보였으며, top-k의 설정에 따라 recall에 차이가 관찰됨
- 따라서 downstream task(예: LLM 응답 생성)의 목적에 맞게 rerank top-k를 조절할 필요 있음

### 결론 및 제안
- Cohere Reranker를 기준으로 응답 품질(LM 응답 정확도 등)을 평가하여 최적의 rerank top-k 설정을 결정하는 후속 실험이 필요

In [107]:
import os
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm
tqdm.pandas()


import sys
sys.path.append('../code/graphParser')
sys.path.append('../code/ragas_custom')
from rateLimit import handle_rate_limits
from retrieve.sparse import BM25
from retrieve.config import generate_retriever_configs
from evaluation.retrieve import combine_hybrid_results, evaluate_metrics

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

from ragas.testset.graph import KnowledgeGraph

import cohere
import requests
from flashrank import Ranker, RerankRequest

In [5]:
dataset = pd.read_csv('../data/rag/self_reflection.csv')
dataset = dataset.iloc[:, :5]
dataset['reference_contexts'] = dataset['reference_contexts'].apply(lambda x : eval(x))

In [6]:
dataset.head(2)

,user_input,reference_contexts,reference,synthesizer_name,reference_contexts_section
0,바벨 잡는 방법과 앉아받기 동작의 올바른 수행을 결합하여 최적의 리프팅 기술을 설명해줘.,"[바벨을 잡는 방법에는 크게 오버그립(over grip), 언더그립(under gr...","바벨 잡는 방법에는 오버그립, 언더그립, 리버스그립, 훅그립이 있으며, 각각의 그립...",multi_hop_abstract_query_synthesizer,"['Ⅲ. 역도경기 기술의 구조와 훈련법', 'Ⅲ. 역도경기 기술의 구조와 훈련법']"
1,"스포츠 심리학자들은 선수의 성과 향상에 어떤 역할을 하며, 그들의 개인 문제에 대한...",[성공적인 상\n담 진행을 위해서 상담사는 내담자의 감정에 공감할 수 있어야 한다....,스포츠 심리학자들은 운동선수의 성과를 향상시키기 위해 여러 가지 역할을 수행합니다....,multi_hop_abstract_query_synthesizer,"['II. 역도의 스포츠 과학적 원리', 'II. 역도의 스포츠 과학적 원리']"


In [15]:
configs = generate_retriever_configs(
    k_values=[20], 
    analyzers=["bm25_kiwi_pos"],
    hybrid_alphas=[40],
    fetch_k=[],
    lambda_mult=[],
    score_threshold=[0.1]
)

config = configs[2]

In [20]:
kg = KnowledgeGraph.load('../data/rag/kg.json')

documents = [Document(page_content=node.properties['page_content'],
                      metadata=node.properties['document_metadata'])
                       for node in kg.nodes]

In [25]:
texts = [node.properties['page_content'] for node in kg.nodes]

kiwi_pos = BM25(k=10, type='kiwi_pos')
kiwi_pos.from_texts(texts)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embeddings)

In [32]:
dataset['precompute_dense'] = dataset['user_input'].progress_apply(lambda x: db.similarity_search_with_score(x, k=20))
dataset['precompute_dense'] = dataset['precompute_dense'].apply(lambda x : [doc.__dict__['page_content'] for doc, score in x if score >= config['dense_params']['score_threshold']])
dataset['precompute_sparse_bm25_kiwi_pos'] = dataset['user_input'].progress_apply(lambda x: kiwi_pos.search(x))
dataset['retrieved_contexts'] = combine_hybrid_results(dataset['precompute_dense'], dataset['precompute_sparse_bm25_kiwi_pos'], config['alpha'], config['k'])
dataset['need_retrieve'] = dataset.apply(lambda x : False if len(set(x['retrieved_contexts']).intersection(set(x['reference_contexts']))) == len(x['reference_contexts']) else True, axis=1)

100%|██████████| 56/56 [00:00<00:00, 609.58it/s]


In [42]:
dataset.head(2)

,user_input,reference_contexts,reference,synthesizer_name,reference_contexts_section,precompute_dense,precompute_sparse_bm25_kiwi_pos,retrieved_contexts,need_retrieve
0,바벨 잡는 방법과 앉아받기 동작의 올바른 수행을 결합하여 최적의 리프팅 기술을 설명해줘.,"[바벨을 잡는 방법에는 크게 오버그립(over grip), 언더그립(under gr...","바벨 잡는 방법에는 오버그립, 언더그립, 리버스그립, 훅그립이 있으며, 각각의 그립...",multi_hop_abstract_query_synthesizer,"['Ⅲ. 역도경기 기술의 구조와 훈련법', 'Ⅲ. 역도경기 기술의 구조와 훈련법']","[기본기 개념을 발달시키는 것은 필수적이며, 바를 들어 올리는 가장 중요한 요\n소...",[순간적으로 무거운 물체를 들어 올리는 데에는 근력 외에도 강인한 정신력이 요구\n...,[순간적으로 무거운 물체를 들어 올리는 데에는 근력 외에도 강인한 정신력이 요구\n...,True
1,"스포츠 심리학자들은 선수의 성과 향상에 어떤 역할을 하며, 그들의 개인 문제에 대한...",[성공적인 상\n담 진행을 위해서 상담사는 내담자의 감정에 공감할 수 있어야 한다....,스포츠 심리학자들은 운동선수의 성과를 향상시키기 위해 여러 가지 역할을 수행합니다....,multi_hop_abstract_query_synthesizer,"['II. 역도의 스포츠 과학적 원리', 'II. 역도의 스포츠 과학적 원리']",[스포츠심리학은 스포츠에 참여하는 개인과 개인의 행동을 과학적으로 탐구하고\n그 지...,[스포츠심리학을 연구하고 스포츠심리학의 연구 성과를 스포츠 현장에 적용하는\n스포츠...,[스포츠심리학을 연구하고 스포츠심리학의 연구 성과를 스포츠 현장에 적용하는\n스포츠...,False


## 1. Cohere

In [43]:
co = cohere.ClientV2()

# dataset['cohere'] = dataset.progress_apply(lambda x : co.rerank(model='rerank-v3.5', query=x['user_input'], documents=x['retrieved_contexts']), axis=1)
dataset['cohere_contexts'] = dataset.apply(lambda x : [result.index for result in x['cohere'].results], axis=1)
dataset['cohere_contexts'] = dataset.apply(lambda x: [x['retrieved_contexts'][i] for i in x['cohere_contexts']], axis=1)
# dataset.to_csv('../data/rag/rerank_dataset.csv', index=False)

V2RerankResponse(id='f8fcd3d8-38b1-43cf-a0c4-79141770eeb6', results=[V2RerankResponseResultsItem(document=None, index=3, relevance_score=0.8029425), V2RerankResponseResultsItem(document=None, index=7, relevance_score=0.7730283), V2RerankResponseResultsItem(document=None, index=14, relevance_score=0.7610953), V2RerankResponseResultsItem(document=None, index=18, relevance_score=0.73459625), V2RerankResponseResultsItem(document=None, index=4, relevance_score=0.69573146), V2RerankResponseResultsItem(document=None, index=19, relevance_score=0.6388584), V2RerankResponseResultsItem(document=None, index=0, relevance_score=0.60049355), V2RerankResponseResultsItem(document=None, index=2, relevance_score=0.5324014), V2RerankResponseResultsItem(document=None, index=8, relevance_score=0.529337), V2RerankResponseResultsItem(document=None, index=1, relevance_score=0.529045), V2RerankResponseResultsItem(document=None, index=11, relevance_score=0.5278769), V2RerankResponseResultsItem(document=None, ind

## 2. Flash

In [78]:
reranker = Ranker(model_name='ms-marco-MultiBERT-L-12', cache_dir='/opt')

dataset['flash'] = dataset['retrieved_contexts'].apply(lambda x : [{'id': i, 'text': doc} for i, doc in enumerate(x)])
# dataset['flash'] = dataset.progress_apply(lambda x : RerankRequest(query=x['user_input'], passages=x['flash']), axis=1)
dataset['flash_contexts'] = dataset['flash'].progress_apply(lambda x : reranker.rerank(x))
dataset['flash_contexts'] = dataset['flash_contexts'].apply(lambda x : [value['text'] for value in x])
# dataset.to_csv('../data/rag/rerank_dataset.csv', index=False)

## 3. Jina

In [100]:
header = {'Authorization': f'Bearer {os.getenv('JINA_API_KEY')}'}
url = "https://api.jina.ai/v1/rerank"

# dataset['jina'] = dataset.progress_apply(lambda x : requests.post(url, headers=header, json={'model': 'jina-reranker-v2-base-multilingual', 'query': x['user_input'], 'documents': x['retrieved_contexts']}).json(), axis=1)
dataset['jina_contexts'] = dataset['jina'].apply(lambda x : [result['document']['text'] for result in x['results']])
# dataset.to_csv('../data/rag/rerank_dataset.csv', index=False)

100%|██████████| 56/56 [01:35<00:00,  1.71s/it]


## 결과 분석

In [ ]:
column_list = ['cohere_contexts', 'flash_contexts', 'jina_contexts']
k_list = range(5, 11, 1)

results = []
for column, k in product(column_list, k_list):
    result = evaluate_metrics(dataset[column], dataset['reference_contexts'], k)
    result['reranker'] = column
    result['k'] = k

    results.append(result)

result_df = pd.DataFrame(results)

In [112]:
result_df = pd.DataFrame(results)

In [114]:
result_df.sort_values(by = ['recall', 'ndcg', 'map'], ascending=False)

,ndcg,recall,map,reranker,k
4,0.660362,0.744048,0.719657,cohere_contexts,9
5,0.660362,0.744048,0.719657,cohere_contexts,10
3,0.653770,0.726190,0.722385,cohere_contexts,8
2,0.639953,0.690476,0.743219,cohere_contexts,7
16,0.595816,0.690476,0.659042,jina_contexts,9
17,0.595816,0.690476,0.659042,jina_contexts,10
15,0.585801,0.666667,0.657802,jina_contexts,8
1,0.629004,0.663690,0.762351,cohere_contexts,6
0,0.625104,0.654762,0.761607,cohere_contexts,5
14,0.578892,0.648810,0.671195,jina_contexts,7
